In [21]:
import os
import json
from openai import AzureOpenAI
from azure.identity import ManagedIdentityCredential, get_bearer_token_provider
import os

token_provider = get_bearer_token_provider(
        ManagedIdentityCredential(), "https://cognitiveservices.azure.com/.default"
        )

client = AzureOpenAI(
        azure_ad_token_provider=token_provider, 
        api_version = "2024-05-01-preview",
        azure_endpoint = "https://aitestepm.openai.azure.com/"
        )

model_name = "gpt-4o" # You need to ensure the version of the model you are using supports the function calling feature
client_id = "fe300394-6c65-467e-9aad-96f2637dbc2d"


credential = ManagedIdentityCredential(client_id=client_id)
token = credential.get_token("d07a4aa0-7148-4e4a-9060-36a8fbec0888/.default")

headers = {
    "Authorization": f"Bearer {token.token}",
    "Content-Type": "application/json"
}

In [11]:
def get_function_call(messages, tool_choice="auto"):
    # Define the functions to use
    tools = [
        {
            "type": "function",
            "function": {
                "name": "Call_Embeddings",
                "description": "Calls embeddings Azure function ",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "Service": {
                            "type": "string",
                            "description": "The service to use as a param",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["Service"],
                },
            },
        },
    ]

    # Call the model with the user query (messages) and the functions defined in the functions parameter
    response = client.chat.completions.create(
        model=model_name,
        messages=messages,
        tools=tools,
        tool_choice=tool_choice,
    )

    return response

In [12]:
first_message = [
    {"role": "user", "content": "Call the embedding function with service WaAppAgent"}
]
# 'auto' : Let the model decide what function to call
print("Let the model decide what function to call:")
print(get_function_call(first_message, "auto").choices[0].message)

# 'none' : Don't call any function
print("Don't call any function:")
print(get_function_call(first_message, "none").choices[0].message)

# force a specific function call
print("Force a specific function call:")
print(
    get_function_call(
        first_message,
        tool_choice={"type": "function", "function": {"name": "Call_Embeddings"}},
    )
    .choices[0]
    .message
)

Let the model decide what function to call:
ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_kCzoOEEyHm5jNG8ePlEUNLUH', function=Function(arguments='{"Service":"WaAppAgent"}', name='Call_Embeddings'), type='function')])
Don't call any function:
ChatCompletionMessage(content='Sure! I\'ll proceed with calling the embeddings function with the service "WaAppAgent".', refusal=None, role='assistant', function_call=None, tool_calls=None)
Force a specific function call:
ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_pWxzkHpRlJoe1M5PiWq08cSK', function=Function(arguments='{"Service":"WaAppAgent"}', name='Call_Embeddings'), type='function')])


In [13]:
def Call_Embeddings(Service):
    try:
        api_endpoint = "https://logicappembeddingfunction.azurewebsites.net/api/embeddingfunction?Service=WaAppAgent"
        response = requests.post(api_endpoint, headers=headers)
        return response._content
    except:
        return "Sorry, the call failed"

In [18]:
import inspect


# helper method used to check if the correct arguments are provided to a function
def check_args(function, args):
    sig = inspect.signature(function)
    params = sig.parameters

    # Check if there are extra arguments
    for name in args:
        if name not in params:
            return False
    # Check if the required arguments are provided
    for name, param in params.items():
        if param.default is param.empty and name not in args:
            return False

    return True

In [ ]:
def run_conversation(messages, tools, available_functions):
    # Step 1: send the conversation and available functions to GPT
    response = client.chat.completions.create(
        model=model_name,
        messages=messages,
        tools=tools,
        tool_choice="auto",
    )

    response_message = response.choices[0].message

    # Step 2: check if GPT wanted to call a function
    if response_message.tool_calls:
        print("Recommended Function call:")
        print(response_message.tool_calls[0])
        print()

        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors

        function_name = response_message.tool_calls[0].function.name

        # verify function exists
        if function_name not in available_functions:
            return "Function " + function_name + " does not exist"
        function_to_call = available_functions[function_name]

        # verify function has correct number of arguments
        function_args = json.loads(response_message.tool_calls[0].function.arguments)
        if check_args(function_to_call, function_args) is False:
            return "Invalid number of arguments for function: " + function_name
        function_response = function_to_call(**function_args)

        print("Output of function call:")
        print(function_response)
        print()

        # Step 4: send the info on the function call and function response to GPT

        # adding assistant response to messages
        messages.append(
            {
                "role": response_message.role,
                "function_call": {
                    "name": response_message.tool_calls[0].function.name,
                    "arguments": response_message.tool_calls[0].function.arguments,
                },
                "content": None,
            }
        )

        # adding function response to messages
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response

        print("Messages in second request:")
        for message in messages:
            print(message)
        print()

        second_response = client.chat.completions.create(
            messages=messages,
            model=model_name,
        )  # get a new response from GPT where it can see the function response

        return second_response